In [1]:
"""
A minimal start.
"""
import time


class React: 
    def __init__(self, verbose: bool = False):
        self.start_time: float | None = None
        self.is_started: bool = False
        self.not_finished: bool = True
        self.intermediate_steps: list[dict] = []
        self.verbose = verbose
    
    def add_intermediate_step(self, intermediate_step: dict):
        self.intermediate_steps.append(intermediate_step)
        if self.verbose:
            print(intermediate_step)
            
    def start(self, question: str):
        self.start_time = time.time()
        self.is_started = True
        self.not_finished = True
        self.intermediate_steps = []
        self.add_intermediate_step({"question": question})

react_agent = React(verbose=True)
react_agent.start("What is 2 + 2?")

{'question': 'What is 2 + 2?'}


In [2]:
"""
Adding reasoning step.
"""
import json
import time
from utils import parse_json, get_completion
from prompts import (
    REACT_TOOLS_DESCRIPTION, 
    REACT_VALID_ACTIONS, 
    REACT_JSON_FORMAT, 
    REACT_PROCESS_FORMAT, 
    REACT_INTERMEDIATE_STEPS, 
    REACT_ADDITIONAL_INSTRUCTIONS
)


class React: 
    def __init__(self, verbose: bool = False):
        self.start_time: float | None = None
        self.is_started: bool = False
        self.not_finished: bool = True
        self.intermediate_steps: list[dict] = []
        self.verbose = verbose
    
    @property
    def steps_count(self):
        return int((len(self.intermediate_steps) - 1 ) / 2)
    
    def add_intermediate_step(self, intermediate_step: dict):
        self.intermediate_steps.append(intermediate_step)
        if self.verbose:
            print(intermediate_step)
            
    def start(self, question: str):
        self.start_time = time.time()
        self.is_started = True
        self.not_finished = True
        self.intermediate_steps = []
        self.add_intermediate_step({"question": question})

    def next(self):
        if not self.is_started:
            raise ValueError("React was not started")
        
        if self.verbose:
            print(f"Step {self.steps_count}")
        thought, action, action_input, is_final_answer = self.reason() 
        self.add_intermediate_step({"thought": thought, "action": action, "action_input": action_input})
    
    def reason(self) -> tuple[str, str, str, bool]:
        messages = self.build_messages()
        completion_response = get_completion(messages, model="gpt-4o", temperature=0, max_tokens=256, stop=["</reasoning>"])
        completion = completion_response.choices[0].message.content
        parsed_completion = parse_json(completion)
        thought = parsed_completion["thought"]
        action = parsed_completion["action"]
        action_input = parsed_completion["action_input"]
        is_final_answer = action == "Final Answer"
        return thought, action, action_input, is_final_answer
    
    def build_messages(self) -> list[dict]:
        question = self.intermediate_steps[0]["question"]
        intermediate_steps=json.dumps(self.intermediate_steps[1:])
        system_prompt_message = \
                REACT_TOOLS_DESCRIPTION.format(tools_description="") + \
                REACT_VALID_ACTIONS.format(tools_names="") + \
                REACT_JSON_FORMAT + \
                REACT_PROCESS_FORMAT + \
                REACT_ADDITIONAL_INSTRUCTIONS + \
                REACT_INTERMEDIATE_STEPS.format(question=question, intermediate_steps=intermediate_steps)
        messages = [{ "role": "system", "content": system_prompt_message }]
        return messages
    
react_agent = React(verbose=True)
react_agent.start("What is 2 + 2?")
react_agent.next()

{'question': 'What is 2 + 2?'}
Step 0
{'thought': 'The question is asking for a simple arithmetic calculation. I can solve this directly without any tools.', 'action': 'Final Answer', 'action_input': 4}


In [3]:
"""
Adding tools
"""
import json
import time
from utils import parse_json, get_completion
from prompts import REACT_TOOLS_DESCRIPTION, REACT_VALID_ACTIONS, REACT_JSON_FORMAT, REACT_PROCESS_FORMAT, REACT_INTERMEDIATE_STEPS, REACT_ADDITIONAL_INSTRUCTIONS


class React: 
    def __init__(self, tools: list[callable], verbose: bool = False):
        self.start_time: float | None = None
        self.is_started: bool = False
        self.not_finished: bool = True
        self.intermediate_steps: list[dict] = []
        self.verbose: bool = verbose
        self.tools_dict: dict[str, callable] = {tool.__name__: tool for tool in tools}
        self.tools_description: str = "\n".join([f"{tool_name}: {tool.__doc__}" for tool_name, tool in self.tools_dict.items()])
        self.tools_names: list[str] = list(self.tools_dict.keys())
        if self.verbose:
            print("Initialized agent with tools:")
            print(f"{self.tools_description}")
            print()
    
    @property
    def steps_count(self):
        return (len(self.intermediate_steps) - 1 )
    
    def add_intermediate_step(self, intermediate_step: dict):
        self.intermediate_steps.append(intermediate_step)
        if self.verbose:
            print(intermediate_step)
            
    def start(self, question: str):
        if self.verbose:
            print(f"Starting agent with:")
        self.start_time = time.time()
        self.is_started = True
        self.not_finished = True
        self.intermediate_steps = []
        self.add_intermediate_step({"question": question})

    def next(self):
        if not self.is_started:
            raise ValueError("React was not started")
        if self.verbose:
            print(f"Step {self.steps_count}")
        thought, action, action_input, is_final_answer = self.reason() 
        self.add_intermediate_step({"thought": thought, "action": action, "action_input": action_input})
    
    def reason(self) -> tuple[str, str, str, bool]:
        messages = self.build_messages()
        completion_response = get_completion(messages, model="gpt-4o", temperature=0, max_tokens=256, stop=["</reasoning>"])
        completion = completion_response.choices[0].message.content
        parsed_completion = parse_json(completion)
        thought = parsed_completion["thought"]
        action = parsed_completion["action"]
        action_input = parsed_completion["action_input"]
        is_final_answer = action == "Final Answer"
        return thought, action, action_input, is_final_answer
    
    def build_messages(self) -> list[dict]:
        question = self.intermediate_steps[0]["question"]
        intermediate_steps=json.dumps(self.intermediate_steps[1:])
        system_prompt_message = \
                REACT_TOOLS_DESCRIPTION.format(tools_description=self.tools_description) + \
                REACT_VALID_ACTIONS.format(tools_names=self.tools_names) + \
                REACT_JSON_FORMAT + \
                REACT_PROCESS_FORMAT + \
                REACT_ADDITIONAL_INSTRUCTIONS + \
                REACT_INTERMEDIATE_STEPS.format(question=question, intermediate_steps=intermediate_steps)
        messages = [{ "role": "system", "content": system_prompt_message }]
        return messages
    
def calculator(expression: str) -> float:
    """Evaluates a mathematical expression and returns the result"""
    return eval(expression)
react_agent = React(tools=[calculator], verbose=True)
react_agent.start("What is 2 + 2?")
react_agent.next()
react_agent.next()

Initialized agent with tools:
calculator: Evaluates a mathematical expression and returns the result

Starting agent with:
{'question': 'What is 2 + 2?'}
Step 0
{'thought': 'The question asks for the sum of 2 and 2, which is a simple arithmetic operation. I will use the calculator to ensure accuracy.', 'action': 'calculator', 'action_input': '2 + 2'}
Step 1
{'thought': 'The question asks for the sum of 2 and 2, which is a simple arithmetic operation. I will use the calculator to ensure accuracy.', 'action': 'calculator', 'action_input': '2 + 2'}


In [13]:
"""
Adding action and observation.
"""
import json
import time
from utils import parse_json, get_completion
from prompts import REACT_TOOLS_DESCRIPTION, REACT_VALID_ACTIONS, REACT_JSON_FORMAT, REACT_PROCESS_FORMAT, REACT_INTERMEDIATE_STEPS, REACT_ADDITIONAL_INSTRUCTIONS


class React: 
    def __init__(self, tools: list[callable], verbose: bool = False):
        self.start_time: float | None = None
        self.is_started: bool = False
        self.not_finished: bool = True
        self.intermediate_steps: list[dict] = []
        self.verbose: bool = verbose
        self.tools_dict: dict[str, callable] = {tool.__name__: tool for tool in tools}
        self.tools_description: str = "\n".join([f"{tool_name}: {tool.__doc__}" for tool_name, tool in self.tools_dict.items()])
        self.tools_names: list[str] = list(self.tools_dict.keys())
        if self.verbose:
            print("Initialized agent with tools:")
            print(f"{self.tools_description}")
            print()
            
    @property
    def steps_count(self):
        return int((len(self.intermediate_steps) - 1 ) / 2)
    
    def add_intermediate_step(self, intermediate_step: dict):
        self.intermediate_steps.append(intermediate_step)
        if self.verbose:
            print(intermediate_step)
            
    def start(self, question: str):
        self.start_time = time.time()
        self.is_started = True
        self.not_finished = True
        self.intermediate_steps = []
        self.add_intermediate_step({"question": question})

    def next(self):
        if not self.is_started:
            raise ValueError("React was not started")
        
        if self.verbose:
            print(f"Step {self.steps_count}")
        thought, action, action_input, is_final_answer = self.reason() 
        self.add_intermediate_step({"thought": thought, "action": action, "action_input": action_input})
        observation = self.act(action, action_input)
        self.add_intermediate_step({"observation": observation})
        
    def reason(self) -> tuple[str, str, str, bool]:
        messages = self.build_messages()
        completion_response = get_completion(messages, model="gpt-4o", temperature=0, max_tokens=256, stop=["</reasoning>"])
        completion = completion_response.choices[0].message.content
        parsed_completion = parse_json(completion)
        thought = parsed_completion["thought"]
        action = parsed_completion["action"]
        action_input = parsed_completion["action_input"]
        is_final_answer = action == "Final Answer"
        return thought, action, action_input, is_final_answer
    
    def build_messages(self) -> list[dict]:
        question = self.intermediate_steps[0]["question"]
        intermediate_steps=json.dumps(self.intermediate_steps[1:])
        system_prompt_message = \
                REACT_TOOLS_DESCRIPTION.format(tools_description=self.tools_description) + \
                REACT_VALID_ACTIONS.format(tools_names=self.tools_names) + \
                REACT_JSON_FORMAT + \
                REACT_PROCESS_FORMAT + \
                REACT_ADDITIONAL_INSTRUCTIONS + \
                REACT_INTERMEDIATE_STEPS.format(question=question, intermediate_steps=intermediate_steps)
        messages = [{ "role": "system", "content": system_prompt_message }]
        return messages
    
    def act(self, action: str, action_input: dict | str) -> str:
        tool_func = self.tools_dict[action]
        if isinstance(action_input, dict):
            tool_result = tool_func(**action_input)
        else:
            tool_result = tool_func(action_input)
        return tool_result
    
def calculator(expression: str) -> float:
    """Evaluates a mathematical expression and returns the result"""
    return eval(expression)
react_agent = React(tools=[calculator], verbose=True)
react_agent.start("What is 2 + 2?")
react_agent.next()
react_agent.next()

Initialized agent with tools:
calculator: Evaluates a mathematical expression and returns the result

{'question': 'What is 2 + 2?'}
Step 0
{'thought': 'The question asks for the sum of 2 and 2, which is a simple arithmetic operation. I will use the calculator to ensure accuracy.', 'action': 'calculator', 'action_input': '2 + 2'}
{'observation': 4}
Step 1
{'thought': 'The observation confirms the result of the arithmetic operation. I now have enough information to provide the final answer.', 'action': 'Final Answer', 'action_input': 4}


KeyError: 'Final Answer'

In [9]:
"""
Add finish step
"""
import json
import time
from utils import parse_json, get_completion
from prompts import REACT_TOOLS_DESCRIPTION, REACT_VALID_ACTIONS, REACT_JSON_FORMAT, REACT_PROCESS_FORMAT, REACT_INTERMEDIATE_STEPS, REACT_ADDITIONAL_INSTRUCTIONS


class React: 
    def __init__(self, tools: list[callable], verbose: bool = False):
        self.is_started: bool = False
        self.start_time: float | None = None
        self.end_time: float | None = None
        self.intermediate_steps: list[dict] = []
        self.verbose: bool = verbose
        self.tools_dict: dict[str, callable] = {tool.__name__: tool for tool in tools}
        self.tools_description: str = "\n".join([f"{tool_name}: {tool.__doc__}" for tool_name, tool in self.tools_dict.items()])
        self.tools_names: list[str] = list(self.tools_dict.keys())
        if self.verbose:
            print("Initialized agent with tools:")
            print(f"{self.tools_description}")
            print()
    @property
    def steps_count(self):
        return int((len(self.intermediate_steps) - 1 ) / 2)
    
    @property
    def last(self):
        return self.intermediate_steps[-1] if self.intermediate_steps else None
    
    def add_intermediate_step(self, intermediate_step: dict):
        self.intermediate_steps.append(intermediate_step)
        if self.verbose:
            print(intermediate_step)
            
    def start(self, question: str):
        self.start_time = time.time()
        self.is_started = True
        self.intermediate_steps = []
        self.add_intermediate_step({"question": question})

    def next(self):
        if not self.is_started:
            raise ValueError("React was not started")
        
        if self.verbose:
            print(f"Step {self.steps_count}")
        thought, action, action_input, is_final_answer = self.reason() 
        self.add_intermediate_step({"thought": thought, "action": action, "action_input": action_input})
        if is_final_answer:
            self.finish()
        else:
            observation = self.act(action, action_input)
            self.add_intermediate_step({"observation": observation})

            
    def reason(self) -> tuple[str, str, str, bool]:
        messages = self.build_messages()
        completion_response = get_completion(messages, model="gpt-4o", temperature=0, max_tokens=256, stop=["</reasoning>"])
        completion = completion_response.choices[0].message.content
        parsed_completion = parse_json(completion)
        thought = parsed_completion["thought"]
        action = parsed_completion["action"]
        action_input = parsed_completion["action_input"]
        is_final_answer = action == "Final Answer"
        return thought, action, action_input, is_final_answer
    
    def build_messages(self) -> list[dict]:
        question = self.intermediate_steps[0]["question"]
        intermediate_steps=json.dumps(self.intermediate_steps[1:])
        system_prompt_message = \
                REACT_TOOLS_DESCRIPTION.format(tools_description=self.tools_description) + \
                REACT_VALID_ACTIONS.format(tools_names=self.tools_names) + \
                REACT_JSON_FORMAT + \
                REACT_PROCESS_FORMAT + \
                REACT_ADDITIONAL_INSTRUCTIONS + \
                REACT_INTERMEDIATE_STEPS.format(question=question, intermediate_steps=intermediate_steps)
        messages = [{ "role": "system", "content": system_prompt_message }]
        return messages
    
    def act(self, action: str, action_input: dict | str) -> str:
        tool_func = self.tools_dict[action]
        if isinstance(action_input, dict):
            tool_result = tool_func(**action_input)
        else:
            tool_result = tool_func(action_input)
        return tool_result
    
    def finish(self):
        self.add_intermediate_step({"answer": self.last["action_input"]})
        self.end_time = time.time()
        
def calculator(expression: str) -> float:
    """Evaluates a mathematical expression and returns the result"""
    return eval(expression)

react_agent = React(tools=[calculator], verbose=True)
react_agent.start("What is 2 + 2?")
react_agent.next()
react_agent.next()

Initialized agent with tools:
calculator: Evaluates a mathematical expression and returns the result

{'question': 'What is 2 + 2?'}
Step 0
{'thought': 'The question asks for the sum of 2 and 2, which is a simple arithmetic operation. I will use the calculator to ensure accuracy.', 'action': 'calculator', 'action_input': '2 + 2'}
{'observation': 4}
Step 1
{'thought': 'The observation confirms the result of the arithmetic operation. I now have enough information to provide the final answer.', 'action': 'Final Answer', 'action_input': 4}
{'answer': 4}


In [5]:
"""
Add loop support
"""
import json
import time
from utils import parse_json, get_completion
from prompts import REACT_TOOLS_DESCRIPTION, REACT_VALID_ACTIONS, REACT_JSON_FORMAT, REACT_PROCESS_FORMAT, REACT_INTERMEDIATE_STEPS, REACT_ADDITIONAL_INSTRUCTIONS


class React: 
    def __init__(self, tools: list[callable], verbose: bool = False):
        self.start_time: float | None = None
        self.end_time: float | None = None
        self.is_started: bool = False
        self.not_finished: bool = True
        self.finish_reason = None
        self.intermediate_steps: list[dict] = []
        self.verbose: bool = verbose
        self.tools_dict: dict[str, callable] = {tool.__name__: tool for tool in tools}
        self.tools_description: str = "\n".join([f"{tool_name}: {tool.__doc__}" for tool_name, tool in self.tools_dict.items()])
        self.tools_names: list[str] = list(self.tools_dict.keys())
        if self.verbose:
            print("Initialized agent with tools:")
            print(f"{self.tools_description}")
            print()
            
    @property
    def steps_count(self):
        return int((len(self.intermediate_steps) - 1 ) / 2)
    @property
    def last(self):
        return self.intermediate_steps[-1] if self.intermediate_steps else None
    
    def add_intermediate_step(self, intermediate_step: dict):
        self.intermediate_steps.append(intermediate_step)
        if self.verbose:
            print(intermediate_step)
            
    def start(self, question: str):
        self.start_time = time.time()
        self.is_started = True
        self.not_finished = True
        self.intermediate_steps = []
        self.add_intermediate_step({"question": question})

    def next(self):
        if not self.is_started:
            raise ValueError("React was not started")
        if not self.not_finished:
            raise ValueError("React is already finished. You can start again.")
        if self.verbose:
            print(f"Step {self.steps_count}")
        thought, action, action_input, is_final_answer = self.reason() 
        self.add_intermediate_step({"thought": thought, "action": action, "action_input": action_input})
        if is_final_answer:
            self.finish()
        else:
            observation = self.act(action, action_input)
            self.add_intermediate_step({"observation": observation})

            
    def reason(self) -> tuple[str, str, str, bool]:
        messages = self.build_messages()
        completion_response = get_completion(messages, model="gpt-4o", temperature=0, max_tokens=256, stop=["</reasoning>"])
        completion = completion_response.choices[0].message.content
        parsed_completion = parse_json(completion)
        thought = parsed_completion["thought"]
        action = parsed_completion["action"]
        action_input = parsed_completion["action_input"]
        is_final_answer = action == "Final Answer"
        return thought, action, action_input, is_final_answer
    
    def build_messages(self) -> list[dict]:
        question = self.intermediate_steps[0]["question"]
        intermediate_steps=json.dumps(self.intermediate_steps[1:])
        system_prompt_message = \
                REACT_TOOLS_DESCRIPTION.format(tools_description=self.tools_description) + \
                REACT_VALID_ACTIONS.format(tools_names=self.tools_names) + \
                REACT_JSON_FORMAT + \
                REACT_PROCESS_FORMAT + \
                REACT_ADDITIONAL_INSTRUCTIONS + \
                REACT_INTERMEDIATE_STEPS.format(question=question, intermediate_steps=intermediate_steps)
        messages = [{ "role": "system", "content": system_prompt_message }]
        return messages
    
    def act(self, action: str, action_input: dict | str) -> str:
        tool_func = self.tools_dict[action]
        if isinstance(action_input, dict):
            tool_result = tool_func(**action_input)
        else:
            tool_result = tool_func(action_input)
        return tool_result
    
    def finish(self):
        self.not_finished = False
        self.finish_reason = "final answer"
        self.add_intermediate_step({"answer": self.last["action_input"]})
        self.end_time = time.time()
        
def calculator(expression: str) -> float:
    """Evaluates a mathematical expression and returns the result"""
    return eval(expression)

react_agent = React(tools=[calculator], verbose=True)
react_agent.start("What is 2 + 2?")
while react_agent.not_finished:
    react_agent.next()

Initialized agent with tools:
calculator: Evaluates a mathematical expression and returns the result

{'question': 'What is 2 + 2?'}
Step 0
{'thought': 'The question asks for the sum of 2 and 2, which is a simple arithmetic operation. I will use the calculator to ensure accuracy.', 'action': 'calculator', 'action_input': '2 + 2'}
{'observation': 4}
Step 1
{'thought': 'The observation confirms the result of the arithmetic operation. I now have enough information to provide the final answer.', 'action': 'Final Answer', 'action_input': 4}
{'answer': 4}


In [6]:
"""
Add max_steps, finish_reason, duration and token_usage
"""
import json
import time
from typing import Literal

import openai
from utils import parse_json, get_completion
from prompts import REACT_TOOLS_DESCRIPTION, REACT_VALID_ACTIONS, REACT_JSON_FORMAT, REACT_PROCESS_FORMAT, REACT_INTERMEDIATE_STEPS, REACT_ADDITIONAL_INSTRUCTIONS


class React: 
    def __init__(self, tools: list[callable], max_steps: int = 10, verbose: bool = False):
        self.max_steps = max_steps
        self.verbose: bool = verbose
        self.tools_dict: dict[str, callable] = {tool.__name__: tool for tool in tools}
        self.tools_description: str = "\n".join([f"{tool_name}: {tool.__doc__}" for tool_name, tool in self.tools_dict.items()])
        self.tools_names: list[str] = list(self.tools_dict.keys())
        
        self.start_time: float | None = None
        self.end_time: float | None = None
        self.is_started: bool = False
        self.not_finished: bool = True
        self.finish_reason: Literal["final answer", "max_steps_reached"] | None = None
        self.intermediate_steps: list[dict] = []
        self.completion_records: list[openai.types.completion.Completion] = []

        if self.verbose:
            print("Initialized agent with tools:")
            print(f"{self.tools_description}")
            print()
            
    @property
    def steps_count(self):
        return int((len(self.intermediate_steps) - 1 ) / 2)
    @property
    def last(self):
        return self.intermediate_steps[-1] if self.intermediate_steps else None
    @property
    def is_max_steps_reached(self):
        return self.steps_count >= self.max_steps
    
    @property
    def duration(self):
        if not self.start_time or not self.end_time:
            return None
        return round((self.end_time - self.start_time)*1000, 3)
    
    @property
    def token_usage(self):
        usage = {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}
        for res in self.completion_records:
            usage['prompt_tokens'] += res.usage.prompt_tokens
            usage['completion_tokens'] += res.usage.completion_tokens
            usage['total_tokens'] += res.usage.total_tokens
        return usage
    
    def add_intermediate_step(self, intermediate_step: dict):
        self.intermediate_steps.append(intermediate_step)
        if self.verbose:
            print(intermediate_step)
            
    def start(self, question: str):
        if self.verbose:
            print(f"Starting agent with:")
        self.start_time = time.time()
        self.is_started = True
        self.not_finished = True
        self.intermediate_steps = []
        self.add_intermediate_step({"question": question})

    def next(self):
        if not self.is_started:
            raise ValueError("React was not started")
        
        if self.verbose:
            print(f"Step {self.steps_count}")
        thought, action, action_input, is_final_answer = self.reason() 
        self.add_intermediate_step({"thought": thought, "action": action, "action_input": action_input})
        if is_final_answer:
            self.finish()
        else:
            observation = self.act(action, action_input)
            self.add_intermediate_step({"observation": observation})
            if self.is_max_steps_reached:
                self.finish()
            
    def reason(self) -> tuple[str, str, str, bool]:
        messages = self.build_messages()
        completion_response = get_completion(messages, model="gpt-4o", temperature=0, max_tokens=256, stop=["</reasoning>"])
        self.completion_records.append(completion_response)
        completion = completion_response.choices[0].message.content
        parsed_completion = parse_json(completion)
        thought = parsed_completion["thought"]
        action = parsed_completion["action"]
        action_input = parsed_completion["action_input"]
        is_final_answer = action == "Final Answer"
        return thought, action, action_input, is_final_answer
    
    def build_messages(self) -> list[dict]:
        question = self.intermediate_steps[0]["question"]
        intermediate_steps=json.dumps(self.intermediate_steps[1:])
        system_prompt_message = \
                REACT_TOOLS_DESCRIPTION.format(tools_description=self.tools_description) + \
                REACT_VALID_ACTIONS.format(tools_names=self.tools_names) + \
                REACT_JSON_FORMAT + \
                REACT_PROCESS_FORMAT + \
                REACT_ADDITIONAL_INSTRUCTIONS + \
                REACT_INTERMEDIATE_STEPS.format(question=question, intermediate_steps=intermediate_steps)
        messages = [{ "role": "system", "content": system_prompt_message }]
        return messages
    
    def act(self, action: str, action_input: dict | str) -> str:
        tool_func = self.tools_dict[action]
        if isinstance(action_input, dict):
            tool_result = tool_func(**action_input)
        else:
            tool_result = tool_func(action_input)
        return tool_result
    
    def finish(self):
        self.not_finished = False
        if not self.is_max_steps_reached:
            self.finish_reason = "final answer"
            self.add_intermediate_step({"answer": self.last["action_input"]})
        else:
            self.finish_reason = "max_steps_reached"
        self.end_time = time.time()
        
def calculator(expression: str) -> float:
    """Evaluates a mathematical expression and returns the result"""
    return eval(expression)

react_agent = React(tools=[calculator], verbose=True)
react_agent.start("What is 2 + 2?")
while react_agent.not_finished:
    react_agent.next()
print(f"Finish reason: {react_agent.finish_reason}")
print(f"Duration: {react_agent.duration} ms")
print(f"Token Usage: {react_agent.token_usage}")

Initialized agent with tools:
calculator: Evaluates a mathematical expression and returns the result

Starting agent with:
{'question': 'What is 2 + 2?'}
Step 0
{'thought': 'The question asks for the sum of 2 and 2, which is a simple arithmetic operation. I will use the calculator to ensure accuracy.', 'action': 'calculator', 'action_input': '2 + 2'}
{'observation': 4}
Step 1
{'thought': 'The observation confirms the result of the arithmetic operation. I now have enough information to provide the final answer.', 'action': 'Final Answer', 'action_input': 4}
{'answer': 4}
Finish reason: final answer
Duration: 2063.474 ms
Token Usage: {'prompt_tokens': 710, 'completion_tokens': 97, 'total_tokens': 807}
